In [ ]:
!pip install --upgrade huggingface_hub
!pip install --upgrade fireworks-ai
#!pip install telebot
!pip install python-telegram-bot --upgrade
!pip install pyTelegramBotAPI

In [2]:
import telebot
import pandas as pd
import random
from huggingface_hub import InferenceClient
import logging
from collections import deque
from telebot import types
from PIL import Image
import numpy as np
logging.basicConfig(level=logging.INFO)

In [ ]:
!git clone https://github.com/frdaria/elfo_bot.git

In [5]:
df = pd.read_csv('elfo_bot/prompts_fandom.csv', on_bad_lines='skip')
chunks = pd.read_csv('elfo_bot/chunks_kinopoisk.csv', on_bad_lines='skip')
vectors = np.load('/content/elfo_bot/bg3_m3_kinopoisk_chunks.npy')

In [6]:
client_gemma = InferenceClient(
    provider="hf-inference",
    api_key=api_key,)
# client_picture = InferenceClient(
#     provider="fal-ai",
#     api_key=api_key,)
# client_mistral = InferenceClient(
#     provider="fireworks-ai",
#     api_key=api_key,
# )

In [8]:
prompt_system_dialogue = '''Ты - волшебный эльф по имени Элфо! Ты - добрый писатель, ты пишешь приличный, дружелюбный и захватывающий контент на русском языке. Ты используешь много эмодзи! Твои тексты просты для чтения, полны диалогов, юмора и шуток! Твоя специализация - фанфики! Ты могу писать тексты в стиле фэнтези про знаменитостей и популярных персонажей книг и сериалов. Ты всегда включаешь рандомного популярного персонажа в сюжет. Когда пользователь дает тебе тему истории, ты начинаешь со слов "Однажды..." и заканчиваешь словами "Продолжим читать?". Если пользователь задает тебе вопрос, не связанный с темой истории, ты отвечаешь: 'Я просто Элфо! Какую историю ты хочешь почитать?'''

# def dialogue_model(user_question: str, context: list) -> str:
#     messages = [
#                 {
#                   "role": "system",
#                   "content": prompt_system_dialogue
#                 },
#                ]
#     if context:
#          messages.extend(context)

#     question = [{'role':'user',
#                  'content': user_question}]
#     messages.extend(question)
#     completion = client_mistral.chat.completions.create(
#       model="mistralai/Mistral-Small-24B-Instruct-2501",
# 	    messages=messages,
#   	  max_tokens=200,
#     )
#     return completion.choices[0].message.content


def update_context(context, user_m, answer):
    new_data = [{'role': 'user', 'content': user_m},
                {'role': 'system', 'content': answer}]
    context.extend(new_data)
    return context

def get_random_prompt():
    prompts = [
        "Жил-был дракон, который любил чай...",
        "Однажды в далёкой галактике...",
        "В маленькой деревне жил волшебник, который боялся магии..."
    ]
    return random.choice(prompts)

# def get_fandom_prompt(fandom):
#     prompts = {
#         "🪄 Гарри Поттер": "Напиши историю о Гарри Поттере, который обнаружил новое заклинание...",
#         "🧙 Властелин Колец": "Напиши историю о Фродо, который нашёл новое кольцо...",
#         "⭐ Звёздные Войны": "Напиши историю о Люке Скайуокере, который встретил нового дроида..."
#     }
#     return prompts.get(fandom, "Фандом не найден.")

def get_fandom_prompt(df, fandom):
    if fandom in df.columns:
        prompts = df[fandom].dropna().tolist()
        if prompts:
            return random.choice(prompts)
        else:
            return "Для этого фандома нет промптов."
    else:
        return "Фандом не найден."


def process_dialogue_step(user_message, context):
        try:
            answer = gemma_generate(user_message, list(context))
        except Exception as e:
            logging.error(f"Ошибка в dialogue_model: {e}")
            return None, None
        if not answer:
            logging.error("Пустой ответ от dialogue_model")
            return None, None
        context = update_context(context, user_message, answer)
        logging.info(f"Updated context: {context}")
        return answer, context

#sys_prompt_gemma = """Ты волшебный Эльф, который пишет волшебные фанфики на русском языке. Твоя задача: 1) по заданному контексту написать историю. 2) Написать промпт для иллюстрации к этой истории. Используй шаблон: 'История:...Иллюстрация:...'Всегда следуй этому шаблону!!!"""

def gemma_generate(user_question: str, context: list) -> str:
    messages=[{
                "role": "system",
                "content": [{"type": "text", "text": prompt_system_dialogue}]
              }]
    if context:
         messages.extend(context)

    question = [{"role":"user",
                 "content": user_question}]
    messages.extend(question)

    completion = client_gemma.chat.completions.create(
                model="google/gemma-3-27b-it",
                max_tokens=200,
                messages = messages)

    gemma_answer = completion.choices[0].message.content.replace('\n\n', '').replace('##', '')
    return gemma_answer

def gemma_create_prompt(story) -> str:
    messages=[{
                "role": "user",
                "content": [{"type": "text", "text": f'Read the text and create a SIMPLE and SHORT prompt to illustrate the plot of the story. The prompt must be in English. Do not give any comments, your ouput must consist of the promplt only!!! Story: {story}'}]
              }]
    completion = client_gemma.chat.completions.create(
                model="google/gemma-3-27b-it",
                max_tokens=50,
                messages = messages)

    description = completion.choices[0].message.content.replace('\n\n', '').replace('##', '')
    return description

def picture_generate(picture_prompt):
    # output is a PIL.Image object
    image = client_picture.text_to_image(f'mystical style inspired by old fairy tales books, flat design. {picture_prompt}',
    model="stabilityai/stable-diffusion-3.5-large",)
    return image

In [8]:
d = gemma_create_prompt('''Однажды... в далёом королевстве Вальдемар, где магические арканы и древние заклятья плелись в ткани повседневной жизни, в тюрьму "Стальной Зверь" попал маг по имени Эйдан. Его обвинили в utilisationе запрещённой магии, и теперь он был заключён в пыточные казематы этой неприступной крепости. Но Эйдан не был бы магом, если бы не знал, как выбраться из любой ситуации! 🔥🏰✨

В ночь, когда луна скрылась за облаками, Эйдан начал свой план побега. Он использовал свои магические знания, чтобы сотворить мощное заклятие, которое открыло тяжелые двери его камеры. Через узкие окно он выбрался на наружную стену, где встретил неожиданного союзника -
Продолжать?''')
d

'A mage imprisoned in a fortress must escape using magic.'

In [9]:
# Ограничиваем контекст последними n сообщениями
MAX_CONTEXT_LENGTH = 7
context_q = deque(maxlen=MAX_CONTEXT_LENGTH)
dialog_check = 0


# @bot.message_handler(commands=["очистить"])
# def clear_context(message):
#     global context_q
#     context_q.clear()  # Очищаем контекст
#     bot.reply_to(message, "Контекст очищен. Начнём с чистого листа!")

fandoms = ["🌹💌 Евгений Онегин", "🧙✨ Властелин Колец", "👑⚔️ Игра престолов", "🔥🐲 Dragon Age", "🦹🏻❤️ Алые сердца Корё"]

# def create_picture_keyboard():
#     keyboard = [
#         [types.KeyboardButton("Изобразить")]
#     ]
#     return types.ReplyKeyboardMarkup(keyboard, resize_keyboard=True)

# def create_picture_keyboard():
#     keyboard = [
#         [types.KeyboardButton("Изобразить")]
#     ]
#     return types.ReplyKeyboardMarkup(keyboard)

def create_picture_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_fandom = types.KeyboardButton("Изобразить")
    keyboard.add(button_fandom)
    return keyboard

# Создаем клавиатуру
def create_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_fandom = types.KeyboardButton("📚 Выбрать фандом")
    keyboard.add(button_fandom)
    return keyboard

def create_fandom_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_eo = types.KeyboardButton("🌹💌 Евгений Онегин")
    button_lotr = types.KeyboardButton("🧙✨ Властелин Колец")
    button_ip = types.KeyboardButton("👑⚔️ Игра престолов")
    button_da = types.KeyboardButton("🔥🐲 Dragon Age")
    button_core = types.KeyboardButton("🦹🏻❤️ Алые сердца Корё")
    keyboard.add(button_eo, button_lotr, button_ip, button_da, button_core)
    return keyboard


# @bot.message_handler(func=lambda message: message.text == "Изобразить")
# def command_for_pic(message):
#     bot.reply_to(message, reply_markup=create_picture_keyboard())

@bot.message_handler(func=lambda message: message.text == "Изобразить")
def handle_picture_request(message):
    global context_q
    if context_q:
        last_answer = context_q[-1]['content']
        if context_q[-1]['role'] == 'system':
            pic_prompt = gemma_create_prompt(last_answer)
        else:
            pic_prompt = context_q[-2]['content'] if len(context_q) > 1 else ""
    else:
        pic_prompt = ""
    try:
        image = picture_generate(pic_prompt)
        image.save('generated_image.jpg')
        bot.send_photo(message.chat.id, open('generated_image.jpg', 'rb'))
        bot.send_message(message.chat.id, "Выберите фандом:", reply_markup=create_keyboard())
    except Exception as e:
        print(f"Ошибка генерации: {e}")
        bot.reply_to(message, "Не удалось создать изображение 😔")
            # Создаем поток байтов
            # buffer = BytesIO()
            # image.save(buffer, format='JPEG')
            # buffer.seek(0)

           # Отправляем изображение как байты
            #bot.send_photo(message.chat.id, buffer)

@bot.message_handler(func=lambda message: message.text == "📚 Выбрать фандом")
def choose_fandom(message):
    bot.reply_to(message, "Выбери фандом:", reply_markup=create_fandom_keyboard())

@bot.message_handler(func=lambda message: message.text in fandoms)
def handle_fandom(message):
    global context_q
    global df
    user_id = message.from_user.id
    fandom = message.text

    # Получаем промпт для выбранного фандома
    prompt = get_fandom_prompt(df, fandom)

    if dialog_check == 1:
      answer, context_q = process_dialogue_step(prompt, context_q)
    if answer is not None:
        bot.reply_to(message, f"{answer}\nПродолжать?", reply_markup=create_picture_keyboard())

@bot.message_handler(func=lambda message: True)
def answer_the_question(message):
    global context_q
    global dialog_check
    user_message = message.text

    # Проверка на команду завершения
    if user_message.lower() in ["стоп", "хватит", "закончим"]:
        bot.reply_to(message, "Хорошо, давай закончим. Если захочешь ещё историй, просто напиши!")
        context_q.clear()  # Очищаем контекст
        dialog_check = 0
        return

    if dialog_check == 0:
        bot.reply_to(message, "Привет! Я Элфо! Какую историю ты хочешь почитать?", reply_markup=create_keyboard())
        dialog_check = 1
        return

    if dialog_check == 1:
      answer, context_q = process_dialogue_step(user_message, context_q)
    if answer is not None:
            bot.reply_to(message, answer, reply_markup=create_keyboard())
            bot.reply_to(message, "Что будет дальше?")

    return

In [10]:
bot.polling(none_stop=True)

In [25]:
from io import BytesIO
from PIL import Image

In [10]:
def send_message_with_picture_keyboard(chat_id, text):
    bot.send_message(chat_id=chat_id, text=text, reply_markup=create_picture_keyboard())

In [ ]:
from telebot import types
from collections import deque
import logging
import random


# Инициализация контекста
MAX_CONTEXT_LENGTH = 5
user_contexts = {}
dialog_check = 0

# Создаем клавиатуру
def create_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_random = types.KeyboardButton("🎲 Рандомный сюжет от Элфо")
    button_fandom = types.KeyboardButton("📚 Выбрать фандом")
    button_continue = types.KeyboardButton("➡️ Продолжить историю")
    keyboard.add(button_random, button_fandom, button_continue)
    return keyboard

# Клавиатура для выбора фандома
def create_fandom_keyboard():
    keyboard = types.ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    button_hp = types.KeyboardButton("🪄 Гарри Поттер")
    button_lotr = types.KeyboardButton("🧙 Властелин Колец")
    button_sw = types.KeyboardButton("⭐ Звёздные Войны")
    keyboard.add(button_hp, button_lotr, button_sw)
    return keyboard

# Функции для промптов
def get_random_prompt():
    prompts = [
        "Жил-был дракон, который любил чай...",
        "Однажды в далёкой галактике...",
        "В маленькой деревне жил волшебник, который боялся магии..."
    ]
    return random.choice(prompts)

def get_fandom_prompt(fandom):
    prompts = {
        "🪄 Гарри Поттер": "Напиши историю о Гарри Поттере, который обнаружил новое заклинание...",
        "🧙 Властелин Колец": "Напиши историю о Фродо, который нашёл новое кольцо...",
        "⭐ Звёздные Войны": "Напиши историю о Люке Скайуокере, который встретил нового дроида..."
    }
    return prompts.get(fandom, "Фандом не найден.")

# Управление контекстом
def get_user_context(user_id):
    if user_id not in user_contexts:
        user_contexts[user_id] = deque(maxlen=MAX_CONTEXT_LENGTH)
    return user_contexts[user_id]

# Обработка нажатий на кнопки
@bot.message_handler(func=lambda message: message.text == "🎲 Рандомный сюжет от Элфо")
def random_story(message):
    prompt = get_random_prompt()
    bot.reply_to(message, prompt)

@bot.message_handler(func=lambda message: message.text == "📚 Выбрать фандом")
def choose_fandom(message):
    bot.reply_to(message, "Выбери фандом:", reply_markup=create_fandom_keyboard())

@bot.message_handler(func=lambda message: message.text in ["🪄 Гарри Поттер", "🧙 Властелин Колец", "⭐ Звёздные Войны"])
def handle_fandom(message):
    user_id = message.from_user.id
    fandom = message.text

    # Получаем промпт для выбранного фандома
    prompt = get_fandom_prompt(fandom)

    # Отправляем запрос к модели
    try:
        answer = dialogue_model(prompt, list(get_user_context(user_id)))
    except Exception as e:
        logging.error(f"Ошибка в dialogue_model: {e}")
        bot.reply_to(message, "Произошла ошибка. Попробуйте ещё раз.")
        return

    if not answer:
        logging.error("Пустой ответ от dialogue_model")
        bot.reply_to(message, "Кажется, Элфо заснул... Попробуйте ещё раз.")
        return

    # Отправляем ответ пользователю
    bot.reply_to(message, answer)

    # Обновляем контекст
    context_q = get_user_context(user_id)
    context_q.append({"user": prompt, "assistant": answer})

    # Возвращаем основную клавиатуру
    bot.reply_to(message, "Что будет дальше?", reply_markup=create_keyboard())

@bot.message_handler(func=lambda message: message.text == "➡️ Продолжить историю")
def continue_story(message):
    user_id = message.from_user.id
    context_q = get_user_context(user_id)
    if context_q:
        bot.reply_to(message, "Продолжи историю:")
    else:
        bot.reply_to(message, "Сначала выбери сюжет или фандом!", reply_markup=create_keyboard())

# Основная логика
@bot.message_handler(func=lambda message: True)
def answer_the_question(message):
    global dialog_check

    user_id = message.from_user.id
    context_q = get_user_context(user_id)

    def process_dialogue_step(user_message, context):
        try:
            answer = dialogue_model(user_message, list(context))
        except Exception as e:
            logging.error(f"Ошибка в dialogue_model: {e}")
            bot.reply_to(message, "Произошла ошибка. Попробуйте ещё раз.")
            return None, None

        if not answer:
            logging.error("Пустой ответ от dialogue_model")
            bot.reply_to(message, "Кажется, Элфо заснул... Попробуйте ещё раз.")
            return None, None

        bot.reply_to(message, answer)
        context.append({"user": user_message, "assistant": answer})
        logging.info(f"Updated context: {context}")

        return answer, context

    user_message = message.text

    if user_message.lower() in ["очистить", "стоп", "хватит"]:
        context_q.clear()
        dialog_check = 0
        bot.reply_to(message, "Контекст очищен. Начнём с чистого листа!", reply_markup=create_keyboard())
        return

    if dialog_check == 0:
        bot.reply_to(message, "Привет! Я Элфо! Какую историю ты хочешь почитать?", reply_markup=create_keyboard())
        dialog_check = 1
        return

    if dialog_check == 1:
        answer, context_q = process_dialogue_step(user_message, context_q)

        if answer is not None:
            bot.reply_to(message, answer)
            bot.reply_to(message, "Что будет дальше?", reply_markup=create_keyboard())
        return

# Запуск бота
bot.polling()

In [ ]:
bot.polling(none_stop=True)

In [ ]:



# # @bot.message_handler(commands=['start'])
# # def greet_user(message):
# #   bot.send_message(message.chat.id, "")

# import logging
# logging.basicConfig(level=logging.INFO)

# dialog_check = 0
# context_q = []
# question = ""
# @bot.message_handler(func=lambda message: True)
# def answer_the_question(message):

#   global dialog_check
#   global context_q
#   global question

#   if dialog_check == 0:
#     bot.reply_to(message, "Привет! Я Элфо! Какую историю, ты хочешь почитать?")
#     dialog_check = 1
#     return

#   if dialog_check == 1:
#     user_message = message.text
#     try:
#       answer = dialogue_model(user_message, context_q)
#     except Exception as e:
#       logging.error(f"Ошибка в dialogue_model: {e}")
#       bot.reply_to(message, "Произошла ошибка. Попробуйте ещё раз.")
#       return


#     bot.reply_to(message, answer)

#     if not answer:
#       logging.error("Пустой ответ от dialogue_model")
#       bot.reply_to(message, "Кажется, Элфо заснул... Попробуйте ещё раз.")
#       return

#     context_q = update_context(context_q, user_message, answer)

#     logging.info(f"Updated context: {context_q}")
#     dialog_check = 2
#     return

#   if dialog_check == 2:
#     user_message = message.text
#     answer = dialogue_model(user_message, context_q)
#     bot.reply_to(message, answer)
#     ## обновляем контекст
#     context_q = update_context(context_q, user_message, answer)
#     print(context_q)
#     dialog_check = 0
#     return

In [ ]:
## вот тут все хорошо работало:

# @bot.message_handler(func=lambda message: True)
# def answer_the_question(message):
#     global context_q
#     global dialog_check

#     def process_dialogue_step(user_message, context):
#         try:
#             answer = dialogue_model(user_message, list(context))  # Преобразуем deque в список
#         except Exception as e:
#             logging.error(f"Ошибка в dialogue_model: {e}")
#             bot.reply_to(message, "Произошла ошибка. Попробуйте ещё раз.")
#             return None, None

#         if not answer:
#             logging.error("Пустой ответ от dialogue_model")
#             bot.reply_to(message, "Кажется, Элфо заснул... Попробуйте ещё раз.")
#             return None, None

#         bot.reply_to(message, answer)
#         context = update_context(context, user_message, answer)
#         logging.info(f"Updated context: {context}")

#         return answer, context

#     user_message = message.text

#     # Проверка на команду завершения
#     if user_message.lower() in ["стоп", "хватит", "закончим"]:
#         bot.reply_to(message, "Хорошо, давай закончим. Если захочешь ещё историй, просто напиши!")
#         context_q.clear()  # Очищаем контекст
#         return

#     if dialog_check == 0:
#         bot.reply_to(message, "Привет! Я Элфо! Какую историю ты хочешь почитать?")
#         dialog_check = 1
#         return

#     # Обработка шага диалога
#     if dialog_check == 1:
#       answer, context_q = process_dialogue_step(user_message, context_q)


#     if context_q:  # Если ответ успешно сгенерирован
#         bot.reply_to(message, "Что будет дальше?😜")  # Подсказка для продолжения

#     return